# Import

In [2]:
import re
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
import pandas as pd
import numpy as np

# For data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# For building the neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Suppress TensorFlow warnings (optional)
import os
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
import xgboost as xgb

# Neural Networks
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import joblib

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Resultados

## Datos 2024 independientes

In [9]:
Independientes2024 = pd.read_csv('../Datos/Procesados/Independientes2024.csv')
Conteo_Farms = pd.read_csv('../Datos/Procesados/FarmCantidad.csv')

In [10]:
xgb_model = joblib.load('../Datos/Resultados/Modelos/xgb_model.pkl')

In [11]:
new_input_data = Independientes2024[Independientes2024["Año Censal"] == 2024]
new_input_data = new_input_data.dropna()

In [12]:
new_input_data['Economic_Units_Per_Capita'] = new_input_data['UE Unidades económicas'] / (new_input_data['Población total'])
new_input_data['Health_Units_Per_Capita'] = new_input_data['UE Sector 62 Servicios de salud y de asistencia social'] / (new_input_data['Población total'])

In [13]:
new_input_data = new_input_data.replace([np.inf, -np.inf], np.nan).dropna()

### Preprocesar

In [14]:
# Load preprocessing objects
imputer = joblib.load('../Datos/Resultados/Modelos/Preprocessing/imputer.pkl')
scaler = joblib.load('../Datos/Resultados/Modelos/Preprocessing/scaler.pkl')

# If you have a list of skewed features saved during training
with open('/content/drive/MyDrive/Dataton/Datos/Resultados/Modelos/Preprocessing/skewed_features.pkl', 'rb') as f:
    skewed_features = joblib.load(f)

# Load the list of features used in training
with open('/content/drive/MyDrive/Dataton/Datos/Resultados/Modelos/Preprocessing/features.pkl', 'rb') as f:
    features = joblib.load(f)

new_data  = new_input_data
# 4. Preprocess the New Data

# a. Extract features
X_new = new_data[features].copy()

# b. Impute missing values
X_imputed_new = pd.DataFrame(imputer.transform(X_new), columns=X_new.columns)

# c. Handle skewed features (apply log1p to the same features as in training)
for feature in skewed_features:
    if feature in X_imputed_new.columns:
        X_imputed_new[feature] = np.log1p(X_imputed_new[feature])

# Copy the DataFrame to avoid modifying the original
X_fe_new = X_imputed_new.copy()

# Validate data and handle infinite or NaN values
X_fe_new.replace([np.inf, -np.inf], np.nan, inplace=True)
X_fe_new.fillna(0, inplace=True)

# f. Feature Scaling
X_scaled_new = pd.DataFrame(scaler.transform(X_fe_new), columns=X_fe_new.columns)

# Ensure the columns match the training data
missing_cols = set(features) - set(X_scaled_new.columns)
for col in missing_cols:
    X_scaled_new[col] = 0  # or np.nan

# Reorder columns to match training data
X_scaled_new = X_scaled_new[features]

### Predecir

In [15]:
# 5. Generate Predictions
# a. Predict using XGBoost model
xgb_pred_new = xgb_model.predict(X_scaled_new)
# b. Predict using CatBoost model


# Add predictions to the new_data DataFrame
new_data['xgb_prediction'] = xgb_pred_new

In [16]:
new_data

,Año Censal,Entidad,Municipio,Edad mediana,Grado promedio de escolaridad de la población de 15 y más años,Hogares censales,Población de 5 años y más que asiste a la escuela,Población derechohabiente a servicios de salud,Población en hogares censales,Población en hogares familiares,...,UE Subsector 621 Servicios médicos de consulta externa y servicios relacionados,UE Subsector 622 Hospitales,UE Subsector 623 Residencias de asistencia social y para el cuidado de la salud,total_area,Prod_Bruta_Per_Capita,%_Pob_Derechohabiente,Densidad_Poblacional,Economic_Units_Per_Capita,Health_Units_Per_Capita,xgb_prediction
4,2024,aguascalientes,aguascalientes,29.2,11.248276,293060.8,263300.0,838275.4,1007159.0,960245.0,...,2624.8,35.6,41.4,1.166359e+09,29674.557677,83.015483,0.865756,0.016300,0.002849,442.317230
9,2024,aguascalientes,asientos,25.2,9.050524,13556.0,14415.6,45828.8,53894.4,52601.6,...,25.4,0.0,1.0,5.433743e+08,3146.418404,84.941135,0.099294,0.008481,0.000549,11.361500
14,2024,aguascalientes,cosio,26.2,9.584578,4125.8,4713.6,15364.6,17781.4,17458.2,...,9.2,0.0,0.0,1.283835e+08,9126.827568,86.399523,0.138516,0.009638,0.000562,5.865078
19,2024,aguascalientes,el llano,27.6,8.974304,5632.4,5488.0,16605.6,21139.4,20468.4,...,5.2,0.0,0.0,5.040510e+08,4851.747219,76.654203,0.042978,0.009112,0.000379,5.202770
24,2024,aguascalientes,jesus maria,27.6,10.861858,37377.2,39790.2,112653.6,142119.6,137353.4,...,108.2,1.0,4.4,4.998951e+08,16033.148300,79.297446,0.284189,0.012894,0.001014,51.405891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7506,2024,zacatecas,valparaiso,31.6,8.981166,9255.0,7838.6,29666.2,31994.2,30126.8,...,52.8,0.0,0.0,5.647397e+09,6512.009515,92.371451,0.005687,0.015992,0.001831,12.719076
7511,2024,zacatecas,villa garcia,27.2,9.703139,5220.8,5219.8,16120.8,20038.6,19590.4,...,34.4,0.0,0.0,3.382305e+08,2271.857555,80.493724,0.059212,0.012852,0.002207,12.826389
7516,2024,zacatecas,villa de cos,29.6,8.178325,9433.8,8341.6,29572.8,34828.8,33842.4,...,45.2,0.0,0.0,6.597039e+09,6217.299444,85.123629,0.005266,0.016724,0.001376,12.489609
7521,2024,zacatecas,villanueva,32.6,8.626008,9531.4,7682.8,26421.6,32279.2,30828.0,...,47.4,-1.2,0.0,2.156506e+09,5557.631572,81.489797,0.015035,0.024186,0.001561,13.764839


### Datos DENUE 2024


In [21]:
denue2024 = pd.read_csv('/content/drive/MyDrive/Dataton/Datos/Procesados/FarmCantidad2024.csv')

In [22]:
farm_counters = pd.merge(new_data, denue2024, left_on=['Año Censal','Entidad', 'Municipio'], right_on=['Año Censal','entidad', 'municipio'], how='outer').dropna().drop(columns=['entidad', 'municipio','num_units'])

In [23]:
farm_counters

,Año Censal,Entidad,Municipio,Edad mediana,Grado promedio de escolaridad de la población de 15 y más años,Hogares censales,Población de 5 años y más que asiste a la escuela,Población derechohabiente a servicios de salud,Población en hogares censales,Población en hogares familiares,...,UE Subsector 622 Hospitales,UE Subsector 623 Residencias de asistencia social y para el cuidado de la salud,total_area,Prod_Bruta_Per_Capita,%_Pob_Derechohabiente,Densidad_Poblacional,Economic_Units_Per_Capita,Health_Units_Per_Capita,xgb_prediction,num_units_municipio
0,2024,aguascalientes,aguascalientes,29.2,11.248276,293060.8,263300.0,838275.4,1007159.0,960245.0,...,35.6,41.4,1.166359e+09,29674.557677,83.015483,0.865756,0.016300,0.002849,442.317230,352.0
1,2024,aguascalientes,asientos,25.2,9.050524,13556.0,14415.6,45828.8,53894.4,52601.6,...,0.0,1.0,5.433743e+08,3146.418404,84.941135,0.099294,0.008481,0.000549,11.361500,16.0
2,2024,aguascalientes,cosio,26.2,9.584578,4125.8,4713.6,15364.6,17781.4,17458.2,...,0.0,0.0,1.283835e+08,9126.827568,86.399523,0.138516,0.009638,0.000562,5.865078,6.0
3,2024,aguascalientes,el llano,27.6,8.974304,5632.4,5488.0,16605.6,21139.4,20468.4,...,0.0,0.0,5.040510e+08,4851.747219,76.654203,0.042978,0.009112,0.000379,5.202770,2.0
4,2024,aguascalientes,jesus maria,27.6,10.861858,37377.2,39790.2,112653.6,142119.6,137353.4,...,1.0,4.4,4.998951e+08,16033.148300,79.297446,0.284189,0.012894,0.001014,51.405891,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,2024,zacatecas,valparaiso,31.6,8.981166,9255.0,7838.6,29666.2,31994.2,30126.8,...,0.0,0.0,5.647397e+09,6512.009515,92.371451,0.005687,0.015992,0.001831,12.719076,16.0
1498,2024,zacatecas,villa garcia,27.2,9.703139,5220.8,5219.8,16120.8,20038.6,19590.4,...,0.0,0.0,3.382305e+08,2271.857555,80.493724,0.059212,0.012852,0.002207,12.826389,10.0
1499,2024,zacatecas,villa de cos,29.6,8.178325,9433.8,8341.6,29572.8,34828.8,33842.4,...,0.0,0.0,6.597039e+09,6217.299444,85.123629,0.005266,0.016724,0.001376,12.489609,14.0
1500,2024,zacatecas,villanueva,32.6,8.626008,9531.4,7682.8,26421.6,32279.2,30828.0,...,-1.2,0.0,2.156506e+09,5557.631572,81.489797,0.015035,0.024186,0.001561,13.764839,14.0


### Diferencia

In [25]:
farm_counters['Difference'] = farm_counters['xgb_prediction'] - farm_counters['num_units_municipio']

farm_counters[['xgb_prediction', 'num_units_municipio', 'Difference']].sort_values(by='Difference', ascending=False)

,xgb_prediction,num_units_municipio,Difference
966,1040.541016,874.0,166.541016
991,286.918213,159.0,127.918213
692,637.608765,525.0,112.608765
84,235.941589,136.0,99.941589
0,442.317230,352.0,90.317230
...,...,...,...
629,1321.650635,1471.0,-149.349365
526,478.015594,647.0,-168.984406
149,1343.382324,1517.0,-173.617676
14,674.320679,856.0,-181.679321


In [28]:
positive_count = (farm_counters['Difference'] > 0).sum()
negative_count = (farm_counters['Difference'] < 0).sum()

print(f"Positive differences: {positive_count}")
print(f"Negative differences: {negative_count}")

Positive differences: 776
Negative differences: 699


In [26]:
resultados = farm_counters.rename(columns={'num_units_municipio': 'num_farms_real'})

In [27]:
resultados.to_csv('/content/drive/MyDrive/Dataton/Datos/Resultados/Resultados2.csv', index=False)

In [29]:
resultados

,Año Censal,Entidad,Municipio,Edad mediana,Grado promedio de escolaridad de la población de 15 y más años,Hogares censales,Población de 5 años y más que asiste a la escuela,Población derechohabiente a servicios de salud,Población en hogares censales,Población en hogares familiares,...,UE Subsector 623 Residencias de asistencia social y para el cuidado de la salud,total_area,Prod_Bruta_Per_Capita,%_Pob_Derechohabiente,Densidad_Poblacional,Economic_Units_Per_Capita,Health_Units_Per_Capita,xgb_prediction,num_farms_real,Difference
0,2024,aguascalientes,aguascalientes,29.2,11.248276,293060.8,263300.0,838275.4,1007159.0,960245.0,...,41.4,1.166359e+09,29674.557677,83.015483,0.865756,0.016300,0.002849,442.317230,352.0,90.317230
1,2024,aguascalientes,asientos,25.2,9.050524,13556.0,14415.6,45828.8,53894.4,52601.6,...,1.0,5.433743e+08,3146.418404,84.941135,0.099294,0.008481,0.000549,11.361500,16.0,-4.638500
2,2024,aguascalientes,cosio,26.2,9.584578,4125.8,4713.6,15364.6,17781.4,17458.2,...,0.0,1.283835e+08,9126.827568,86.399523,0.138516,0.009638,0.000562,5.865078,6.0,-0.134922
3,2024,aguascalientes,el llano,27.6,8.974304,5632.4,5488.0,16605.6,21139.4,20468.4,...,0.0,5.040510e+08,4851.747219,76.654203,0.042978,0.009112,0.000379,5.202770,2.0,3.202770
4,2024,aguascalientes,jesus maria,27.6,10.861858,37377.2,39790.2,112653.6,142119.6,137353.4,...,4.4,4.998951e+08,16033.148300,79.297446,0.284189,0.012894,0.001014,51.405891,30.0,21.405891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,2024,zacatecas,valparaiso,31.6,8.981166,9255.0,7838.6,29666.2,31994.2,30126.8,...,0.0,5.647397e+09,6512.009515,92.371451,0.005687,0.015992,0.001831,12.719076,16.0,-3.280924
1498,2024,zacatecas,villa garcia,27.2,9.703139,5220.8,5219.8,16120.8,20038.6,19590.4,...,0.0,3.382305e+08,2271.857555,80.493724,0.059212,0.012852,0.002207,12.826389,10.0,2.826389
1499,2024,zacatecas,villa de cos,29.6,8.178325,9433.8,8341.6,29572.8,34828.8,33842.4,...,0.0,6.597039e+09,6217.299444,85.123629,0.005266,0.016724,0.001376,12.489609,14.0,-1.510391
1500,2024,zacatecas,villanueva,32.6,8.626008,9531.4,7682.8,26421.6,32279.2,30828.0,...,0.0,2.156506e+09,5557.631572,81.489797,0.015035,0.024186,0.001561,13.764839,14.0,-0.235161
